In [ ]:
# Method to calculate the size of the data used for the respective test run
def calculate_data_size(row, df_data_size):
    # Index of the last file used for this test
    last_file_index = row['last_file_index']

    # Filter for all files which were used to perform this test
    relevant_data = df_data_size[df_data_size.index <= last_file_index]

    csv_size = relevant_data['input_data'].sum()
    parquet_size = relevant_data['output_data'].sum()

    return csv_size, parquet_size


In [0]:
import pandas as pd

df_result_aggregated = pd.read_csv(os.path.join(result_folder_path, 'test_result_aggregated.csv'), sep=';', index_col=0)
df_result_non_aggregated = pd.read_csv(os.path.join(result_folder_path, 'test_result_non_aggregated'), sep=';',
                                       index_col=0)

# Combine the results of the aggregated and non aggregated version into one dataframe
df_result = pd.concat([df_result_aggregated, df_result_non_aggregated], axis=1, keys=['aggregated', 'non_aggregated'])

df_data_size = pd.read_csv(os.path.join(result_folder_path, 'data_size.csv'), sep=';', index_col=0)

# Calculate the size of the data used for each test run
df_result[['csv_size', 'parquet_size']] = df_result.apply(lambda row: calculate_data_size(row, df_data_size), axis=1,
                                                          result_type='expand')